## Method of comparing DMI and OCS subscribers

SQL DMI Request

si.FLAG_CONFIRMED - null mean confirmed

SELECT m.NAME AS MVNO_NAME
,s.IMSI,s.MSISDN
,si.IMSI_NUMBER AS S_IMSI,si.PROF_ID
,sp.NAME AS SP_NAME
,s.SIM_APP_VER
,ASCII(si.FLAG_CONFIRMED) AS FLAG
FROM
SUBSCRIBER s, MVNO m, S_IMSI si, SPONSOR sp
WHERE s.MVNO_REF = m.RI
AND s.RI = si.SUBSCRIBER_REF
AND sp.RI = si.SPONSOR_REF
--AND ROWNUM <= 1000
ORDER BY m.NAME,s.IMSI,si.PROF_ID

In [3]:
from os.path import join
import pandas as pd
from pandas import Series, DataFrame
import roamability as rb
import numpy as np

downloads = r'c:\Users\balob\Downloads\REPORT_OCS_DMI_SUBSCRIBERS_COMPARE'
dmi_subs_file_name = 'export_dmi_subs_190502.csv'
ocs_subs_file_name = 'export_ocs_subs_190502.csv'

dict_dmi_flag = {'0': 'Confirmed'
                ,np.nan: 'Confirmed'
                ,'1': 'Not confirmed'
                ,'2': 'Disconnected'}

list_sponsor = [
    ['Cellact','P4_Combined','S2','Combined'],
    ['Cellact','Partner_Combined','S1','Combined'],
    ['Citictell','P4_Combined','S2','Combined'],
    ['Citictell','Partner_Combined','S1','Combined'],
    ['Clay','P4_Clay_Combined','S2','ClayCombined'],
    ['Clay','Partner_Clay_Com','S1','ClayCombined'],
    ['Maxcom','P4_Combined','S2','Combined'],
    ['Maxcom','Partner_Combined','S1','Combined'],
    ['Mobileye','P4_BLOCK','S2','MobileyeCombined'],
    ['Mobileye','Partner_mobileye','S1','MobileyeCombined'],
    ['Mondicon','P4_mon-sky','S2','MondiconMonSky'],
    ['Mondicon','P4_mon_Russia','S2','MondiconMonRus'],
    ['Mondicon','P4_mon_bss','S2','MondiconMonBss'],
    ['Mondicon','P4_mon_euro','S2','MondiconMonEuro'],
    ['Mondicon','P4_mon_world','S2','MondiconMonWorld'],
    ['Mondicon','P4_mondic','S2','MondiconCombined'],
    ['Mondicon','Partner_mon-sky','S1','MondiconMonSky'],
    ['Mondicon','Partner_mon_bss','S1','MondiconMonBss'],
    ['Mondicon','Partner_mon_euro','S1','MondiconMonEuro'],
    ['Mondicon','Partner_mon_rus','S1','MondiconMonRus'],
    ['Mondicon','Partner_mon_worl','S1','MondiconMonWorld'],
    ['Mondicon','Partner_mondic','S1','MondiconCombined'],
    ['Nextel','P4_Combined','S2','Combined'],
    ['Nextel','Partner_Combined','S1','Combined'],
    ['Nextel','Partner_CombNext','S1','NextelCombined'],
    ['Nextel','P4_Comb_Nextel','S2','NextelCombined'],
    ['PuertoSeguro','P4_PS','S2','PuertoSeguroCombined'],
    ['PuertoSeguro','Partner_PS','S1','PuertoSeguroCombined'],
    ['RUSEC','P4_Combined','S2','Combined'],
    ['RUSEC','Partner_Combined','S1','Combined'],
    ['RedT_Partner_Sof','Partner_RedT_Sof','S1','RedTeaPartnerSoft'],
    ['Rewicom','P4_Combined','S2','Combined'],
    ['Rewicom','Partner_Combined','S1','Combined'],
    ['STI','P4_STI_Comb','S2','StiCombined'],
    ['STI','P4_STI_Ping','S2','StiCombinedPing'],
    ['STI','Partner_STI_Comb','S1','StiCombined'],
    ['STI','Partner_STI_Ping','S1','StiCombinedPing'],
    ['TATTELECOM','P4_Combined','S2','Combined'],
    ['TATTELECOM','Partner_Combined','S1','Combined'],
    ['TECHMOB','P4_Combined','S2','Combined'],
    ['TECHMOB','Partner_Combined','S1','Combined'],
    ['Telzar','P4_telzar','S2','TelzarCombined'],
    ['Telzar','PA_Telzar_Soft','S1','TelzarS1Soft'],
    ['Telzar','Partner_telzar','S1','TelzarCombined'],
    ['Telzar_Soft','PA_Telzar_Soft','S1','TelzarS1Soft'],
    ['Tinklabs','P4_tinklabs','S2','TinklabsCombined'],
    ['Tinklabs','Partner_tinklabs','S1','TinklabsCombined'],
    ['WMB_Combined','P4_WMB_Combined','S2','WmbCombined'],
    ['WMB_Combined','Partner_WMB_Comb','S1','WmbCombined'],
    ['Roamability','P4_Combined','S2','Combined'],
    ['Roamability','Partner_Combined','S1','Combined'],
    ['Roamability','Partner_All','S1','RoamabilityS1Soft'],
    ['Roamability','P4_All','S2','RoamabilityS2Soft'],
    ['Partner_Joy_Soft','Partner_Joy_All','S1','JoyS1Soft'],
    ['TECHMOB','P4_All','S2','TechmobS2Soft'],
    ['TECHMOB','Partner_All','S1','TechmobS1Soft'],
    ['Roamability','P4_WeStream','S2','WeStreamS2Soft'],
    ['Roamability','Partner_WeStream','S1','WeStreamS1Soft'],
]

df_prof_book = DataFrame(list_sponsor, columns=['MVNO_NAME', 'SP_NAME', 'SPONSOR', 'PROFILE'])

def print_raw_data(imsi):
    print('Данные DMI:')
    display(df_dmi[df_dmi.IMSI == imsi])
    print('Данные OCS:')
    display(df_ocs[df_ocs.IMSI == imsi])
    print('Объединенные данные DMI и OCS:')
    display(df_ocs_dmi[df_ocs_dmi.IMSI == imsi])

### Получить данные DMI из файла

In [5]:
df_dmi = pd.read_csv(join(downloads, dmi_subs_file_name), dtype='str')
df_dmi.FLAG.fillna('0', inplace=True)
df_dmi.FLAG.replace(dict_dmi_flag, inplace=True)
df_dmi.loc[df_dmi.FLAG == ''] = 'Missing'

df_dmi.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG
0,Cellact,425180020000000,972557012345,425019620010108,1,Partner_Combined,7,Confirmed
1,Cellact,425180020000000,972557012345,260060140068752,2,P4_Combined,7,Confirmed
2,Cellact,425180020000002,NaN,425019620011335,1,Partner_Combined,7,Disconnected


In [6]:
df_dmi.dtypes

MVNO_NAME      object
IMSI           object
MSISDN         object
S_IMSI         object
PROF_ID        object
SP_NAME        object
SIM_APP_VER    object
FLAG           object
dtype: object

### Получить данные OCS

In [2]:
# Получить абонентов OCS (из базы данных)

sql_srt='''
SELECT r.RESELLER_NAME
,a.ACCOUNT_NAME
,si.IMSI
,sm.PHONE_NUMBER
,mi.IMSI AS S_IMSI
,ss.STATUS, ss.START_DATE
,s.subscriber_id AS SUBSCRIBER_ID
FROM SUBSCRIBER_IMSIS si
LEFT JOIN MULTI_IMSI mi ON si.subscriber_id = mi.subscriber_id
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
LEFT JOIN SUBSCRIBERS s ON si.subscriber_id = s.subscriber_id
LEFT JOIN SUBSCRIBER_PHONE_NUMBERS sm ON s.subscriber_id=sm.subscriber_id
LEFT JOIN ACCOUNTS a ON a.account_id = s.account_id
LEFT JOIN RESELLERS r ON r.reseller_id = a.reseller_id
WHERE ss.END_DATE IS NULL
AND mi.END_DATE IS NULL
AND sm.END_DATE IS NULL
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn: # prod connection
    df_ocs = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs.head(3)

,RESELLER_NAME,ACCOUNT_NAME,IMSI,S_IMSI,STATUS,START_DATE,SUBSCRIBER_ID
0,Porto Seguro DMI RT,Porto Seguro Conecta,724540540263237,425019620000023,Active,2016-09-21 00:19:07.907,195602
1,Porto Seguro DMI RT,Porto Seguro Conecta,724540540263250,425019620000025,Active,2016-09-21 16:50:24.880,195604
2,Porto Seguro DMI RT,Porto Seguro Conecta,724540540263257,425019620000031,Active,2016-09-29 21:49:12.187,195609


In [4]:
# Сохранить данные абонентов OCS в файл

df_ocs.to_csv(join(downloads, ocs_subs_file_name), index=False)

In [4]:
# Получить абонентов OCS (из файла)

#df_ocs = pd.read_csv(join(downloads, ocs_subs_file_name), dtype='str')
#df_ocs.head(3)

,RESELLER_NAME,ACCOUNT_NAME,IMSI,S_IMSI,STATUS,START_DATE,SUBSCRIBER_ID
0,Telzar 019 DMI RT,019 Production Account,425191201070391,425019629808689,Active,2017-07-14 00:22:40.250,250023
1,Telzar 019 DMI RT,019 Production Account,425191201070392,425019629808690,Active,2017-07-14 00:22:40.310,250024
2,Telzar 019 DMI RT,019 Production Account,425191201070393,425019629808691,Active,2017-07-14 00:22:40.360,250025


In [8]:
df_ocs.dtypes

RESELLER_NAME            object
ACCOUNT_NAME             object
IMSI                     object
S_IMSI                   object
STATUS                   object
START_DATE       datetime64[ns]
SUBSCRIBER_ID            object
dtype: object

### Объединить и подготовить данные OCS и DMI

In [6]:
list_ocs_status = ['Active', 'Suspended', 'Manually Suspended', 'Auto Suspended', 'Pending', 'Activation Failed']

df_ocs_dmi = pd.merge(df_dmi[df_dmi.MSISDN.notnull()], df_ocs[df_ocs.STATUS.isin(list_ocs_status)],
                      how='outer', on=['IMSI', 'S_IMSI'])
df_ocs_dmi.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE,SUBSCRIBER_ID
0,Cellact,425180020000000,972557012345,425019620010108,1,Partner_Combined,7,Confirmed,Cellact,Cellact Test sim cards,Active,2018-12-03 09:34:31.007,1449894
1,Cellact,425180020000000,972557012345,260060140068752,2,P4_Combined,7,Confirmed,Cellact,Cellact Test sim cards,Active,2018-12-03 09:34:31.007,1449894
2,Cellact,425180020000004,972523726464,425019620012943,1,Partner_Combined,7,Confirmed,Cellact,Cellact Test sim cards,Active,2019-02-26 15:10:45.367,1552817


### Анализ Case 1. DMI-y; OCS-n

In [7]:
# Case 1. DMI-y; OCS-n

df_ocs_n = df_ocs_dmi[(df_ocs_dmi.STATUS.isnull())]
df_ocs_n.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE,SUBSCRIBER_ID
13891,Nextel,724390992819509,5513974156460,425019620003960,1,Partner_CombNext,7,Confirmed,NaN,NaN,NaN,NaT,NaN
13892,Nextel,724390992819509,5513974156460,260060140017975,2,P4_Comb_Nextel,7,Confirmed,NaN,NaN,NaN,NaT,NaN
14057,Nextel,724390993764798,5511947054004,425019620005373,1,Partner_CombNext,7,Confirmed,NaN,NaN,NaN,NaT,NaN


In [8]:
df_ocs_n.MVNO_NAME.value_counts()

Telzar          94
Nextel          12
WMB_Combined     1
Name: MVNO_NAME, dtype: int64

In [13]:
df = df_ocs_n[(df_ocs_n.MVNO_NAME == 'WMB_Combined')].copy()
df

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE,SUBSCRIBER_ID
1417988,WMB_Combined,425019613030889,972540405389,260060140017986,2,P4_WMB_Combined,6,Confirmed,NaN,NaN,NaN,NaT,NaN


In [12]:
print_raw_data('425191202442837')

Данные DMI:


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG
1401605,Telzar,425191202442837,972550871537,425019628890537,1,Partner_telzar,7,Confirmed
1401606,Telzar,425191202442837,972550871537,260060140018014,2,P4_telzar,7,Confirmed


Данные OCS:


,RESELLER_NAME,ACCOUNT_NAME,IMSI,S_IMSI,STATUS,START_DATE,SUBSCRIBER_ID
1326147,Telzar 019 DMI RT,019 Production Account,425191202442837,425019628890537,Active,2018-12-06 13:18:53.247,1540556


Объединенные данные DMI и OCS:


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE,SUBSCRIBER_ID
1385988,Telzar,425191202442837,972550871537,425019628890537,1,Partner_telzar,7,Confirmed,Telzar 019 DMI RT,019 Production Account,Active,2018-12-06 13:18:53.247,1540556
1385989,Telzar,425191202442837,972550871537,260060140018014,2,P4_telzar,7,Confirmed,NaN,NaN,NaN,NaT,NaN


In [14]:
# Выгрузить проблемных абонентов

df_ocs_dmi[df_ocs_dmi.IMSI.isin(df_ocs_n.IMSI.values)
          ].sort_values(by=['MVNO_NAME', 'IMSI']).to_csv(join(downloads, 'OcsMissingImsi.csv'), index=False)

### Методика решения проблемы Case 1. DMI-y; OCS-n

In [57]:
# Выгрузить необходимые данные из OCS

sql_srt='''
SELECT
si.subscriber_id, si.IMSI, COUNT(*) AS NUM
FROM SUBSCRIBER_IMSIS si
LEFT JOIN MULTI_IMSI mi ON si.subscriber_id=mi.subscriber_id
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
WHERE ss.STATUS = 'Active' AND ss.END_DATE IS NULL
AND si.imsi IN ('''\
+ str(df_ocs_n[(df_ocs_n.MVNO_NAME == 'Telzar') & (df_ocs_n.SP_NAME == 'P4_telzar')].IMSI.tolist())[1:-1]\
+ ') GROUP BY si.subscriber_id, si.imsi HAVING COUNT(*) = 1'

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn: # prod connection
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

pd.merge(df_ocs_n[(df_ocs_n.MVNO_NAME == 'Telzar') & (df_ocs_n.SP_NAME == 'P4_telzar')], df, how='inner', on='IMSI')

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE,subscriber_id,NUM
0,Telzar,425191202103584,972506839259,260060140093145,2,P4_telzar,7,NaN,NaN,NaN,NaT,1155237,1
1,Telzar,425191202251043,972552451521,260060140090997,2,P4_telzar,7,NaN,NaN,NaN,NaT,1348001,1
2,Telzar,425191202277272,972552478089,260060140091934,2,P4_telzar,7,NaN,NaN,NaN,NaT,1374230,1


In [120]:
pd.merge(df_ocs_n[(df_ocs_n.MVNO_NAME == 'Telzar') & (df_ocs_n.SP_NAME == 'P4_telzar')], df, how='inner', on='IMSI')\
[['subscriber_id','IMSI','S_IMSI','MVNO_NAME']].to_csv(join(downloads,'Case1.csv'), index=False)

In [ ]:
# Далее по методике c:\W_DATA_ROAM\ПРОЕКТЫ\REPORT_OCS_DMI_SUBSCRIBERS_COMPARE\FIX_MISSED_P4_IMSI\

In [17]:
# Case 1. DMI-y; OCS-n. У абонента нет в OCS ни доного IMSI (частный случай)

df_ocs_dmi[(df_ocs_dmi.IMSI.notnull()) & (df_ocs_dmi.S_IMSI.isnull())].head()

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE,SUBSCRIBER_ID


### Анализ Case 2. DMI-n; OCS-y

In [15]:
# Case 2. DMI-n; OCS-y

df_dmi_n = df_ocs_dmi[(df_ocs_dmi.IMSI.notnull()) & (df_ocs_dmi.MVNO_NAME.isnull())]
df_dmi_n.head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE,SUBSCRIBER_ID
1418617,NaN,99999425019620000102,NaN,425019620000102,NaN,NaN,NaN,NaN,Cloud 9 TAP,Cloud 9 DMI Subscribers,Active,2016-11-10 20:31:56.537,195675
1418618,NaN,99999425019620000104,NaN,425019620000104,NaN,NaN,NaN,NaN,Cloud 9 TAP,Cloud 9 DMI Subscribers,Active,2016-11-10 20:32:18.673,195677
1418619,NaN,99999425019620000105,NaN,425019620000105,NaN,NaN,NaN,NaN,Cloud 9 TAP,Cloud 9 DMI Subscribers,Active,2016-11-10 20:32:28.483,195678


In [16]:
df_dmi_n.groupby([df_dmi_n.RESELLER_NAME, df_dmi_n.ACCOUNT_NAME], as_index=False)['IMSI'].count()

,RESELLER_NAME,ACCOUNT_NAME,IMSI
0,Cloud 9 TAP,Cloud 9 DMI Subscribers,10
1,HMG Int,HMG Int simcards tests,1
2,KnowRoaming IMSI-RANGE TAP,test_kr,1
3,Maxcom RT,Celmax-Soriana,2
4,Maxcom RT,Maxcom Test Sim,4
5,Mobileye,Mobileye test simcards,10
6,Mondicon Relay,Mondicon IMSI Range Real Time (Relay),134
7,Mondicon Relay,Mondicon IMSI Range SIM Cards,1817
8,MultiByte RB Simcards,MultiByte Test Sim cards,1
9,Netmore DMI RT,netmore DMI Subscribers,448


In [23]:
df_dmi_n.to_csv(join(downloads, 'MissedDmi.csv'), index=False)

In [17]:
df_dmi_n[(df_dmi_n.ACCOUNT_NAME == 'netmore DMI Subscribers')].head(3)

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,STATUS,START_DATE,SUBSCRIBER_ID
1418622,NaN,240400000022213,NaN,425019620000387,NaN,NaN,NaN,NaN,Netmore DMI RT,netmore DMI Subscribers,Pending,2016-12-29 12:23:18.227,195962
1418623,NaN,240400000022214,NaN,425019620000388,NaN,NaN,NaN,NaN,Netmore DMI RT,netmore DMI Subscribers,Pending,2016-12-29 12:25:27.333,195963
1418624,NaN,240400000022219,NaN,425019620000393,NaN,NaN,NaN,NaN,Netmore DMI RT,netmore DMI Subscribers,Pending,2016-12-29 12:37:25.200,195968


In [97]:
print_raw_data('425191201621922')

Данные DMI:


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG
607606,Telzar,425191201621922,972559811976,425019620005126,1,Partner_telzar,0,Confirmed
607607,Telzar,425191201621922,972559811976,260060140002337,2,P4_telzar,0,Confirmed


Данные OCS:


,RESELLER_NAME,ACCOUNT_NAME,IMSI,PHONE_NUMBER,S_IMSI,STATUS,START_DATE,SUBSCRIBER_ID
341970,Telzar 019 DMI RT,019 Production Account,425191201621922,972559811976,425019629416872,Active,2018-02-19 18:45:35.337,748137
1384003,Telzar 019 DMI RT,019 Production Account,425191201621922,972559811976,425019620005126,Pending,2018-02-19 14:56:20.407,731260
1384004,Telzar 019 DMI RT,019 Production Account,425191201621922,972559811976,260060140002337,Pending,2018-02-19 14:56:20.407,731260


Объединенные данные DMI и OCS:


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,PHONE_NUMBER,STATUS,START_DATE,SUBSCRIBER_ID
607606,Telzar,425191201621922,972559811976,425019620005126,1,Partner_telzar,0,Confirmed,Telzar 019 DMI RT,019 Production Account,972559811976,Pending,2018-02-19 14:56:20.407,731260
607607,Telzar,425191201621922,972559811976,260060140002337,2,P4_telzar,0,Confirmed,Telzar 019 DMI RT,019 Production Account,972559811976,Pending,2018-02-19 14:56:20.407,731260
1436182,NaN,425191201621922,NaN,425019629416872,NaN,NaN,NaN,NaN,Telzar 019 DMI RT,019 Production Account,972559811976,Active,2018-02-19 18:45:35.337,748137


In [ ]:
# Выгрузить в файл

# Summary
df_dmi_n.groupby([df_dmi_n.RESELLER_NAME, df_dmi_n.ACCOUNT_NAME],
                 as_index=False)['IMSI'].count().to_csv(join(downloads, ''))

### Оценка корректности профайлов в DMI

In [20]:
df_dmi['SPONSOR'] = np.nan
df_dmi.loc[df_dmi.S_IMSI.str.startswith('42501', na=False), ['SPONSOR']] = 'S1'
df_dmi.loc[df_dmi.S_IMSI.str.startswith('26006', na=False), ['SPONSOR']] = 'S2'

In [21]:
df_prof_book.head(3)

,MVNO_NAME,SP_NAME,SPONSOR,PROFILE
0,Cellact,P4_Combined,S2,Combined
1,Cellact,Partner_Combined,S1,Combined
2,Citictell,P4_Combined,S2,Combined


In [22]:
df_sponsor_dmi = df_dmi.groupby(['MVNO_NAME', 'SP_NAME', 'SPONSOR'],
                                as_index=False)['IMSI'].count().sort_values(by=['MVNO_NAME', 'SP_NAME'])
df_sponsor_dmi.head(3)

,MVNO_NAME,SP_NAME,SPONSOR,IMSI
0,Cellact,P4_Combined,S2,5
1,Cellact,Partner_Combined,S1,5
2,Citictell,P4_Combined,S2,3


In [23]:
df_temp = pd.merge(df_sponsor_dmi, df_prof_book, how='left', on=['MVNO_NAME', 'SP_NAME', 'SPONSOR'])
df_temp[df_temp.PROFILE.isnull()]

,MVNO_NAME,SP_NAME,SPONSOR,IMSI,PROFILE
6,Dialoq_Partner_S,Partner_Dialoq_A,S1,12,NaN
7,MB,P4_Combined,S2,2,NaN
8,MB,Partner_Combined,S1,2,NaN
13,Mondicon,P4_Mon-Germ,S2,25,NaN
20,Mondicon,Partner_BLOCK,S1,1118,NaN
27,MultiByte,P4_Combined,S2,1,NaN
28,MultiByte,Partner_Combined,S1,1,NaN
29,Nextel,P4_All,S2,2,NaN
35,RB_fastint_test,Partner_Combined,S1,1,NaN
38,RedT_Partner_Sof,P4_All,S2,15,NaN


In [37]:
# Выгрузить в файл

df_temp.loc[df_temp.PROFILE.isnull(),
            ['MVNO_NAME', 'SP_NAME', 'SPONSOR', 'IMSI']].to_csv(join(downloads, 'DmiIncorrectProfilesSummary.csv'), index=False)

pd.merge(df_dmi,
         df_temp.loc[df_temp.PROFILE.isnull(), ['MVNO_NAME', 'SP_NAME', 'SPONSOR']], how='inner',
         on=['MVNO_NAME', 'SP_NAME', 'SPONSOR']).to_csv(join(downloads, 'DmiIncorrectProfiles.csv'), index=False)

### Поиск дублированных в OCS записей

In [91]:
s_imsi_dublicated = df_ocs.loc[df_ocs.STATUS == 'Active', ['IMSI', 'S_IMSI']].groupby(['IMSI', 'S_IMSI']).size()
s_imsi_dublicated[s_imsi_dublicated.values > 1]

Series([], dtype: int64)

### Поиск абонентов в OCS с тремя и более IMSI

In [98]:
df_temp = df_ocs.groupby('IMSI', as_index=False)['S_IMSI'].count()
df_ocs_three = df_ocs[df_ocs.IMSI.isin(df_temp[df_temp.S_IMSI > 2].IMSI.tolist())]
df_ocs_three.head(3)

,RESELLER_NAME,ACCOUNT_NAME,IMSI,PHONE_NUMBER,S_IMSI,STATUS,START_DATE,SUBSCRIBER_ID
3199,Roamability Test RT,Roamability Test Sim-cards,425019613000536,447781518929,425019613000536,Active,2017-04-12 17:13:02.283,209325
3200,Roamability Test RT,Roamability Test Sim-cards,425019613000536,447781518929,260060149000536,Active,2017-04-12 17:13:02.283,209325
3219,Roamability Test RT,Roamability Test Sim-cards,425019613000585,447781518790,425019613000585,Active,2017-04-13 12:32:09.137,209383


In [112]:
df_ocs_three.groupby(['RESELLER_NAME','ACCOUNT_NAME'])['IMSI'].nunique('IMSI').reset_index()

,RESELLER_NAME,ACCOUNT_NAME,IMSI
0,Maxcom RT,Maxcom Test Sim,2
1,MultiByte RB Simcards,MultiByte Test Sim cards,2
2,Porto Seguro DMI RT,Porto Seguro Conecta,1
3,Roamability Test RT,Antoloy ( Roy's test),1
4,Roamability Test RT,NOC test SIMs,1
5,Roamability Test RT,Roamability Test Sim-cards,11
6,Roamability Test RT,Sure sample cards,4
7,Roamability Test RT,Verint (Roy's test),5
8,Telzar 019 DMI RT,019 Production Account,132
9,TinkLabs RT,Tinklabs Ghana subscribers,3


### IMSI S2 belongs to different subscribers in OCS and DMI

In [122]:
df_temp = df_ocs_dmi.groupby('S_IMSI', as_index=False)['IMSI'].count()
df_temp = df_temp[df_temp.IMSI == 2]
list_spoiled_s2 = df_temp.S_IMSI.tolist()
df_spoiled_real = df_dmi.loc[df_dmi.S_IMSI.isin(list_spoiled_s2), ['IMSI','S_IMSI']]
df_s2_dif_subs = pd.merge(df_ocs, df_spoiled_real, how='inner', on='IMSI', suffixes=['_S1', '_S2'])
df_s2_dif_subs.head()

,RESELLER_NAME,ACCOUNT_NAME,IMSI,PHONE_NUMBER,S_IMSI_S1,STATUS,START_DATE,SUBSCRIBER_ID,S_IMSI_S2
0,Telzar 019 DMI RT,019 Production Account,425191201415465,972559873436,425019629615465,Active,2017-09-28 21:46:53.260,467951,260060140017985
1,Telzar 019 DMI RT,019 Production Account,425191201434329,972559637356,425019629634329,Active,2017-09-28 22:28:50.217,486815,260060140017971
2,Telzar 019 DMI RT,019 Production Account,425191201546116,972559734676,425019629541516,Active,2017-11-20 21:55:02.977,604412,260060140018058
3,Telzar 019 DMI RT,019 Production Account,425191201618744,972559835782,425019629413694,Active,2018-02-19 18:42:49.830,744959,260060140017983
4,Telzar 019 DMI RT,019 Production Account,425191201712251,972559975012,425019629307151,Active,2018-04-21 13:00:46.047,841153,260060140018040


In [137]:
df_s2_dif_subs.groupby(['RESELLER_NAME','ACCOUNT_NAME'], as_index=False)['IMSI'].count()

,RESELLER_NAME,ACCOUNT_NAME,IMSI
0,Telzar 019 DMI RT,019 Production Account,94
1,WMB Limited RT,WMB 5K Simcards order,1


In [134]:
s2_imsi = '260060140017985'
real_imsi_ocs = df_ocs[df_ocs.S_IMSI == s2_imsi].IMSI.values[0]
real_imsi_dmi = df_dmi[df_dmi.S_IMSI == s2_imsi].IMSI.values[0]
print_raw_data(real_imsi_ocs)
print_raw_data(real_imsi_dmi)

Данные DMI:


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG


Данные OCS:


,RESELLER_NAME,ACCOUNT_NAME,IMSI,PHONE_NUMBER,S_IMSI,STATUS,START_DATE,SUBSCRIBER_ID
571163,Netmore DMI RT,netmore DMI Subscribers,240400000038416,46731724416,425019620006694,Active,2018-07-14 20:43:06.700,1090179
571164,Netmore DMI RT,netmore DMI Subscribers,240400000038416,46731724416,260060140017985,Active,2018-07-14 20:43:06.700,1090179


Объединенные данные DMI и OCS:


...

,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG
397582,Telzar,425191201415465,972559873436,425019629615465,1,Partner_telzar,6,Confirmed
397583,Telzar,425191201415465,972559873436,260060140017985,2,P4_telzar,6,Confirmed


Данные OCS:


,RESELLER_NAME,ACCOUNT_NAME,IMSI,PHONE_NUMBER,S_IMSI,STATUS,START_DATE,SUBSCRIBER_ID
159103,Telzar 019 DMI RT,019 Production Account,425191201415465,972559873436,425019629615465,Active,2017-09-28 21:46:53.260,467951


Объединенные данные DMI и OCS:


,MVNO_NAME,IMSI,MSISDN,S_IMSI,PROF_ID,SP_NAME,SIM_APP_VER,FLAG,RESELLER_NAME,ACCOUNT_NAME,PHONE_NUMBER,STATUS,START_DATE,SUBSCRIBER_ID
397582,Telzar,425191201415465,972559873436,425019629615465,1,Partner_telzar,6,Confirmed,Telzar 019 DMI RT,019 Production Account,972559873436,Active,2017-09-28 21:46:53.260,467951
397583,Telzar,425191201415465,972559873436,260060140017985,2,P4_telzar,6,Confirmed,NaN,NaN,NaN,NaN,NaT,NaN
